# BERTopic Tutorial
Grootendorst, M. (2022). BERTopic: Neural topic modeling with a class-based TF-IDF procedure. arXiv preprint arXiv:2203.05794.

https://arxiv.org/pdf/2203.05794.pdf

Document Embedding (BERT) > Dimension Reduction (UMAP) > Clustering (HDBSCAN) > Topic Representation (TF/IDF)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

# Data
For this example, we use a large Kaggle dataset with 20 topics for news headlines from Huffington Post.

https://www.kaggle.com/datasets/rmisra/news-category-dataset

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/EunCheolChoi0123/COMM557Tutorial/refs/heads/main/news_headline_sample.csv'
df  = pd.read_csv(url)
df = df[~df.headline.isna()]
df = df.sample(frac = 0.1)
docs = df.headline.to_list()

In [ ]:
len(docs)

20952

In [ ]:
df.category.value_counts()

,count
category,
POLITICS,3578
WELLNESS,1765
ENTERTAINMENT,1705
TRAVEL,1039
STYLE & BEAUTY,980
PARENTING,894
HEALTHY LIVING,678
FOOD & DRINK,642
QUEER VOICES,636


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


In [ ]:
%%time
# This line took 3 minutes on T4
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

/usr/local/lib/python3.12/dist-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.
2025-09-25 22:42:04,478 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/655 [00:00<?, ?it/s]

2025-09-25 22:42:14,920 - BERTopic - Embedding - Completed ✓
2025-09-25 22:42:14,922 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-25 22:42:52,086 - BERTopic - Dimensionality - Completed ✓
2025-09-25 22:42:52,088 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-25 22:44:48,523 - BERTopic - Cluster - Completed ✓
2025-09-25 22:44:48,536 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-25 22:44:49,000 - BERTopic - Representation - Completed ✓


CPU times: user 3min 18s, sys: 8.34 s, total: 3min 27s
Wall time: 3min 36s


## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. -1 refers to all outliers. We will reduce outliers by assiging documents to topics that has similar embeddings.

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9629,-1_the_to_of_in,"[the, to, of, in, trump, for, and, on, is, video]","[What Does This Election Say About Us?, 15 Thi..."
1,0,522,0_mom_mothers_parenting_moms,"[mom, mothers, parenting, moms, parents, paren...",[There's Only One Thing Moms Want This Mother'...
2,1,467,1_fashion_photos_style_dress,"[fashion, photos, style, dress, week, beauty, ...","[The Models Of Men's Fashion Week, Amanda Seyf..."
3,2,273,2_divorce_dating_relationship_marriage,"[divorce, dating, relationship, marriage, love...",[Dating After Divorce: How To Date A Man With ...
4,3,256,3_recipes_recipe_cheese_cookies,"[recipes, recipe, cheese, cookies, salad, cake...","[Recipe Of The Day: Green Soup (PHOTOS), Recip..."


In [ ]:
# Reduce outliers using the `embeddings` strategy
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings")

In [ ]:
topic_model.update_topics(docs, topics=new_topics)

2025-09-25 22:44:53,178 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,0,652,0_mom_moms_mothers_parenting,"[mom, moms, mothers, parenting, parents, my, k...",[There's Only One Thing Moms Want This Mother'...
1,1,735,1_fashion_photos_style_beauty,"[fashion, photos, style, beauty, photo, dress,...","[The Models Of Men's Fashion Week, Amanda Seyf..."
2,2,343,2_divorce_relationship_love_marriage,"[divorce, relationship, love, marriage, dating...",[Dating After Divorce: How To Date A Man With ...
3,3,390,3_recipes_recipe_cheese_cookies,"[recipes, recipe, cheese, cookies, summer, sal...","[Recipe Of The Day: Green Soup (PHOTOS), Recip..."
4,4,276,4_black_racism_white_confederate,"[black, racism, white, confederate, charlottes...",[NAACP President: The Confederate Flag Should ...


Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('mom', np.float64(0.026627706449222736)),
 ('moms', np.float64(0.01965203867734169)),
 ('mothers', np.float64(0.01919577626563973)),
 ('parenting', np.float64(0.01896616759912484)),
 ('parents', np.float64(0.016928502181051925)),
 ('my', np.float64(0.016215484956660222)),
 ('kids', np.float64(0.01592001475911028)),
 ('parent', np.float64(0.01591116189596788)),
 ('dad', np.float64(0.014669786688645305)),
 ('daughter', np.float64(0.013983479619853157))]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

[89, 122, 249, 94, 176, 170, 1, 214, 0, 9]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can
be used to understand how confident BERTopic is that certain topics can be found in a document.

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.001)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [ ]:
topic_model.update_topics(docs, n_gram_range=(2, 3))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('mothers day', np.float64(0.009992662875528883)),
 ('fathers day', np.float64(0.007927457718162735)),
 ('letter to', np.float64(0.004844549278140042)),
 ('to my', np.float64(0.004731605995089237)),
 ('my daughter', np.float64(0.004533351289254025)),
 ('your kids', np.float64(0.0044363223656770446)),
 ('my son', np.float64(0.003947366881117687)),
 ('letter to my', np.float64(0.0039039073738369495)),
 ('working mom', np.float64(0.002930052990151568)),
 ('how my', np.float64(0.002846665921625414))]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(docs, nr_topics=60)

2025-09-25 22:45:02,717 - BERTopic - Topic reduction - Reducing number of topics
2025-09-25 22:45:02,798 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-25 22:45:04,598 - BERTopic - Representation - Completed ✓
2025-09-25 22:45:04,607 - BERTopic - Topic reduction - Reduced number of topics from 299 to 60


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[0, 17, 56, 13, 0, 10, 1, 0, 4, 21, 14, 6, 11, 1, 0, 3, 15, 41, 6, 29, 0, 5, 39, 5, 4, 23, 30, 4, 33, 12, 1, 4, 39, 18, 0, 0, 3, 17, 17, 46, 33, 27, 23, 14, 17, 3, 15, 14, 1, 25, 12, 18, 11, 38, 1, 0, 4, 0, 1, 13, 0, 35, 0, 20, 38, 26, 18, 4, 32, 13, 3, 29, 11, 2, 35, 26, 6, 19, 0, 0, 41, 15, 14, 11, 0, 11, 22, 9, 3, 31, 0, 12, 1, 1, 32, 0, 0, 5, 16, 10, 18, 1, 0, 12, 34, 7, 0, 6, 7, 3, 45, 21, 0, 39, 5, 2, 19, 1, 7, 40, 8, 0, 3, 15, 2, 6, 0, 0, 10, 26, 0, 3, 17, 26, 41, 0, 28, 2, 10, 40, 2, 2, 9, 10, 0, 2, 0, 13, 0, 25, 15, 5, 1, 7, 0, 17, 2, 20, 3, 0, 9, 16, 34, 0, 24, 0, 56, 25, 11, 0, 21, 7, 5, 1, 1, 39, 17, 30, 9, 1, 27, 17, 19, 3, 7, 4, 1, 26, 2, 8, 0, 36, 33, 12, 1, 0, 9, 33, 39, 2, 1, 13, 0, 19, 26, 19, 20, 0, 0, 45, 16, 6, 11, 2, 0, 0, 20, 16, 5, 0, 43, 2, 8, 0, 32, 9, 1, 4, 0, 18, 13, 0, 12, 0, 6, 9, 0, 9, 4, 10, 45, 0, 6, 13, 15, 0, 7, 0, 26, 9, 36, 18, 39, 29, 17, 17, 7, 31, 42, 11, 8, 11, 5, 20, 25, 26, 5, 29, 4, 1, 21, 5, 20, 39, 18, 5, 3, 5, 5, 2, 9, 34, 0, 16, 8, 13, 0,

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [ ]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[22, 4, 5, 10, 17]

In [ ]:
topic_model.get_topic(similar_topics[0])

[('box office', np.float64(0.013500139644713133)),
 ('weekend box office', np.float64(0.006611532488633585)),
 ('weekend box', np.float64(0.006611532488633585)),
 ('of the', np.float64(0.0058061701816457725)),
 ('at the', np.float64(0.005404219049606495)),
 ('in israel', np.float64(0.00522680803024553)),
 ('at the box', np.float64(0.00522680803024553)),
 ('the box office', np.float64(0.00522680803024553)),
 ('the movies', np.float64(0.005094128724595976)),
 ('syrian refugees', np.float64(0.004981967156456246))]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
topic_model.save("my_model")

2025-09-25 22:45:05,192 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
# Load model
my_model = BERTopic.load("my_model")

# On Your Own
1. Use your collected 1,000 Reddit submissions OR the dataset you collected for your project.
2. Run BERTopic
- Reduce outliers
- Update topics so that it makes the best sense.